# Assessment of Roofs Using Satellite Images

# Part I: Data Preprocessing


In [1]:
import geopandas as gpd
import pandas as pd
import ee # earth engine
from shapely import wkt # for 

### I. Get polygon points
1. Geojson file -> CSV file

In [ ]:
# Load the GeoJSON file
file_path = 'data/sample_geojson_file.geojson'
gdf = gpd.read_file(file_path)

# Define the output CSV file path
output_csv_path = 'data/sample.csv'

# Save the GeoDataFrame as a CSV file, excluding the index
gdf.to_csv(output_csv_path, index=False)

2. CSV file -> correct coordinates format (need to be revised)

In [ ]:
# Define a function to convert WKT to coordinates
def wkt_to_coordinates(wkt_string):
    polygon_geometry = wkt.loads(wkt_string)
    return [[coord[0], coord[1]] for coord in polygon_geometry.exterior.coords]
# Apply the function to the entire column
df['geometry_coordinates'] = df['geometry'].apply(wkt_to_coordinates)

# Drop the original WKT column if needed
df = df.drop(columns=['geometry'])
print(df)

### II. Import dataset (csv version)

1. Use the following chunk for importing processed csv file.
2. Extract the coordinates column and store as a list.

In [2]:
# Load the CSV file
csv_file_path = 'data/sample1.csv'
df = pd.read_csv(csv_file_path)
geometry_coordinates_list = df['geometry_coordinates'].tolist()

# Part II: Finding Images of Roofs

### I. Authenticate and initialize the earth engine api.

In [3]:
ee.Authenticate()
ee.Initialize(project='baxsxasd')

### II. Define the get_image_download_url() function.


In [4]:
def get_image_download_url(polygon_coords, start_date, end_date, vis_params, scale=0.5):
    """
    Generate a direct download URL for an image with visualization parameters.
    
    :param polygon_coords: Coordinates for the polygon defining the area of interest.
    :param start_date: Start date for the image collection filter.
    :param end_date: End date for the image collection filter.
    :param vis_params: Visualization parameters including 'min', 'max', and bands 'palette' if needed.
    :param scale: Scale in meters; 1 meter is typical for NAIP imagery.
    :return: A URL from which the image can be directly downloaded.
    """
    roi = ee.Geometry.Polygon([polygon_coords])
    
    # Filter the NAIP ImageCollection
    naip_collection = ee.ImageCollection('USDA/NAIP/DOQQ') \
                        .filterBounds(roi) \
                        .filterDate(start_date, end_date)
    
    # Get the median image, clip to the ROI, and select RGB bands
    median_image = naip_collection.median().clip(roi).select(['R', 'G', 'B'])
    
    # Generate the download URL with visualization parameters
    url = median_image.getThumbURL({
        'region': roi.getInfo()['coordinates'],
        'scale': scale,
        **vis_params  # Unpack visualization parameters
    })
    
    return url

# Visualization parameters for floating-point data
vis_params = {
    'min': 0.0,
    'max': 255.0,
    'format': 'png'
}

### III. Extract and Store Images
1. Extract the image url links from gee.

In [5]:
for polygon_coords_str in geometry_coordinates_list:
    # Convert the string representation to a list (assuming it's safely formatted)
    polygon_coords = eval(polygon_coords_str)
    
    # Generate the download URL
    download_url = get_image_download_url(polygon_coords, '2022-01-01', '2022-02-15', vis_params, scale=0.5)
    print(download_url)

https://earthengine.googleapis.com/v1/projects/baxsxasd/thumbnails/ce11358b234dcf42e1bdc30ec87aeb3e-feea549902d2170a8a1265f31f7765ea:getPixels
https://earthengine.googleapis.com/v1/projects/baxsxasd/thumbnails/d007e012d5d5ccbd5dc966b0971600d6-f1263c5f71198102388d684635f63dc5:getPixels
https://earthengine.googleapis.com/v1/projects/baxsxasd/thumbnails/bceb4d1cffdfe3a49aff43d584524c5a-0d6fec0007f274166b72027f1eb0a338:getPixels
https://earthengine.googleapis.com/v1/projects/baxsxasd/thumbnails/113dc39f7695b2fbc35ebc330e987e5e-0f215038a38920029ee5e79c088a8faa:getPixels
https://earthengine.googleapis.com/v1/projects/baxsxasd/thumbnails/5787d848d5901893da964229d58a9fd2-9f7e1a3ce6211182fb8067d2a594a821:getPixels
https://earthengine.googleapis.com/v1/projects/baxsxasd/thumbnails/41d334f0f6d396323a595f23be3375a8-99cb6d4f7287cb78ef4a29cfb92ab6ae:getPixels
https://earthengine.googleapis.com/v1/projects/baxsxasd/thumbnails/9a0fe6be41f1441bbda9d1bd1594aa9c-62ec0dc2604892702a817c520b48e213:getPixels

2. Store the images from the links.

3. Store the pixels and image links to the database.

# Part III: Database?

# Part IV: Image Analysis